# December 2018: Advent of Code

## Days 6-10

### Common imports & library functions

In [136]:
from collections import Counter, defaultdict, namedtuple
import doctest
import heapq
import itertools
import math
import numpy as np
import re
import string

### Day 6: Chronal Coordinates

In [132]:
def manhattan_distance(p1, p2):
    """
    >>> manhattan_distance((1, -1), (8, 3))
    11
    """
    a1, a2 = p1
    b1, b2 = p2
    return abs(a1 - b1) + abs(a2 - b2)

def grid_corner(coords_np):
    return tuple(coords_np.max(axis=0))

def parse_coordinates(coords):
    coords_np = []
    for coords_txt in coords:
        x, y = coords_txt.split(',')
        x = int(x)
        y = int(y)
        coords_np.append((x, y))
    return np.array(coords_np)

def print_grid(grid):
    for row in grid:
        print(''.join(row))

PLACEHOLDER = '.'
def make_grid(corner):
    grid = []
    for y in range(corner[1] + 1):
        grid.append([PLACEHOLDER] * (corner[0] + 1))
    return grid
        
def assign_targets(grid, targets):
    for i, coord in enumerate(targets):
        grid[coord[1]][coord[0]] = string.ascii_uppercase[i]

def find_closest_target(grid, targets, coord):
    cell = grid[coord[1]][coord[0]]
    min_distance = float('inf')
    nearest_target = None
    nearest_count = 0
    for i, target in enumerate(targets):
        distance = manhattan_distance(coord, target)
        if distance == min_distance:
            nearest_count += 1
        elif distance < min_distance:
            min_distance = distance
            nearest_target = i
            nearest_count = 1
    return nearest_target if nearest_count == 1 else None 
    
def assign_all_closest_target(grid, targets):
    coords_by_target = defaultdict(list)
    for y in range(len(grid)):
        for x in range(len(grid[0])):
            target = find_closest_target(grid, targets, (x, y))
            if target is not None:
#                 if grid[y][x] == PLACEHOLDER:
#                     grid[y][x] = string.ascii_lowercase[target]
                coords_by_target[target].append((x, y)) 
    return coords_by_target
                
def has_edge_coord(grid, coords):
    for (x, y) in coords:
        if x == 0 or y == 0 or x == len(grid[0]) or y == len(grid):
            return True
    return False

def largest_area(grid, targets):
    coords_by_target = assign_all_closest_target(grid, targets)
    max_area = 0
    for target, coords in coords_by_target.items():
        if has_edge_coord(grid, coords):
            continue
        elif len(coords) > max_area:
            max_area = len(coords)
    return max_area

def largest_area_within_distance(grid, targets, distance):
    coords = [(x, y) for y in range(len(grid)) for x in range(len(grid[0]))]
    return len([c for c in coords if sum(manhattan_distance(c, t) for t in targets) < distance])

In [133]:
# Run unit tests
doctest.testmod()

TestResults(failed=0, attempted=1)

In [135]:
# Final answers
with open('day6_input.txt') as f:
    targets = parse_coordinates(f.read().strip().split('\n'))
    grid = make_grid(grid_corner(targets))
    print('Part 1: ', largest_area(grid, targets))
    print('Part 2: ', largest_area_within_distance(grid, targets, 10000))

Part 1:  4143
Part 2:  35039


### Day 7: The Sum of Its Parts

In [317]:
class DependencyGraph:
    """
    >>> a = DependencyGraph()
    >>> a.add_dependency('A', 'C')
    >>> a.add_dependency('F', 'C')
    >>> a.add_dependency('B', 'A')
    >>> a.add_dependency('D', 'A')
    >>> a.add_dependency('E', 'B')
    >>> a.add_dependency('E', 'D')
    >>> a.add_dependency('E', 'F')
    >>> ''.join(a.pop_satisfied() for _ in range(6))
    'CABDFE'
    """
    def __init__(self):
        self.depends_on = defaultdict(set)
        self.depended_on_by = defaultdict(set)
        self.satisfied = set()
        
    def add_dependency(self, node, depends_on_node):
        self.depends_on[node].add(depends_on_node)
        self.depended_on_by[depends_on_node].add(node)
        if node in self.satisfied:
            self.satisfied.remove(node)
        if depends_on_node not in self.depends_on:
            self.satisfied.add(depends_on_node)
            
    def remove_dependency(self, node, depends_on_node):
        self.depends_on[node].discard(depends_on_node)
        self.depended_on_by[depends_on_node].discard(node)        
        if not self.depends_on[node]:
            self.satisfied.add(node)

    def has_satisfied(self):
        return len(self.satisfied)
            
    def get_satisfied(self):
        return sorted(self.satisfied)
    
    def pop_satisfied(self, node=None):
        node = node or self.get_satisfied()[0]
        self.satisfied.discard(node)
        for other_node in list(self.depended_on_by[node]):
            self.remove_dependency(other_node, node)
        return node
    
    
class DeferredWork:
    def __init__(self, on_complete, completion_time):
        self.time_left = completion_time
        self.on_complete = on_complete
        self.on_complete_called = False
        
    def __repr__(self):
        return 'DeferredWork({}, {})'.format(self.on_complete.__name__, self.time_left)

    def work_on(self, duration=1):
        self.time_left = max(self.time_left - duration, 0)
        if self.is_complete() and not self.on_complete_called:
            self.on_complete_called = True
            self.on_complete()
        return self

    def is_complete(self):
        return self.time_left == 0
        
class TimedDependencyGraph(DependencyGraph):
    def __init__(self, timer_fn):
        super(TimedDependencyGraph, self).__init__()
        self.deferred_satisfied = set()
        self.timer_fn = timer_fn
        
    def timed_has_satisfied(self):
        return len(self.satisfied - self.deferred_satisfied)
        
    def timed_get_satisfied(self):
        return sorted(self.satisfied - self.deferred_satisfied)
        
    def timed_pop_satisfied(self):
        node = self.timed_get_satisfied()[0]
        self.deferred_satisfied.add(node)
        return DeferredWork(lambda: self.pop_satisfied(node), self.timer_fn(node))

def parse_dependency(sentence):
    """
    >>> parse_dependency('Step C must be finished before step A can begin.')
    ('A', 'C')
    """
    depends_on, node = re.match(r'Step (\w) must be finished before step (\w) can begin.', 
                                sentence).groups()
    return (node, depends_on)

def build_dependency_graph(dependencies, timer_fn=None):
    if timer_fn:
        graph = TimedDependencyGraph(timer_fn)
    else:
        graph = DependencyGraph()
    for (node, depends_on) in dependencies:
        graph.add_dependency(node, depends_on)
    return graph

def correct_instruction_order(dependencies):
    graph = build_dependency_graph(dependencies)
    order = []
    while graph.has_satisfied():
        order.append(graph.pop_satisfied())
    return ''.join(order) 

def name_based_timer(base_duration, node):
    """
    >>> name_based_timer(60, 'A')
    61
    >>> name_based_timer(30, 'Z')
    56
    """
    return base_duration + ord(node.lower()) - ord('a') + 1

def time_to_complete(dependencies, workers=5, base_duration=60, debug=False):
    graph = build_dependency_graph(dependencies, 
                                   timer_fn=lambda n: name_based_timer(base_duration, n))
    tasks = []
    time = 0
    while graph.has_satisfied():
        if debug: print('Satisfied:', graph.satisfied)
        if len(tasks) < workers:
            to_assign = min(workers - len(tasks), graph.timed_has_satisfied())
            # Assign more tasks.
            if debug: print('\tAssigning {} more tasks!'.format(to_assign))
            tasks.extend(graph.timed_pop_satisfied() for _ in range(to_assign))
            if debug: print('\tCurrent tasks: {}'.format(tasks))
        step = min(task.time_left for task in tasks)
        tasks = [task for task in tasks if not task.work_on(step).is_complete()]
        time += step
        if debug: print('\tAdvancing to time', time)
    return time

In [321]:
instructions = """Step C must be finished before step A can begin.
Step C must be finished before step F can begin.
Step A must be finished before step B can begin.
Step A must be finished before step D can begin.
Step B must be finished before step E can begin.
Step D must be finished before step E can begin.
Step F must be finished before step E can begin."""
dependencies = [parse_dependency(l) for l in instructions.splitlines()]
assert time_to_complete(dependencies, base_duration=0, debug=True) == 14

Satisfied: {'C'}
	Assigning 1 more tasks!
	Current tasks: [DeferredWork(<lambda>, 3)]
	Advancing to time 3
Satisfied: {'A', 'F'}
	Assigning 2 more tasks!
	Current tasks: [DeferredWork(<lambda>, 1), DeferredWork(<lambda>, 6)]
	Advancing to time 4
Satisfied: {'D', 'F', 'B'}
	Assigning 2 more tasks!
	Current tasks: [DeferredWork(<lambda>, 5), DeferredWork(<lambda>, 2), DeferredWork(<lambda>, 4)]
	Advancing to time 6
Satisfied: {'D', 'F'}
	Assigning 0 more tasks!
	Current tasks: [DeferredWork(<lambda>, 3), DeferredWork(<lambda>, 2)]
	Advancing to time 8
Satisfied: {'F'}
	Assigning 0 more tasks!
	Current tasks: [DeferredWork(<lambda>, 1)]
	Advancing to time 9
Satisfied: {'E'}
	Assigning 1 more tasks!
	Current tasks: [DeferredWork(<lambda>, 5)]
	Advancing to time 14


In [263]:
# Run unit tests
doctest.testmod()

TestResults(failed=0, attempted=13)

In [320]:
# Final answers
with open('day7_input.txt') as instructions:
    dependencies = [parse_dependency(l) for l in instructions]
    print('Part 1: ', correct_instruction_order(dependencies))
    print('Part 2: ', time_to_complete(dependencies))

Part 1:  OKBNLPHCSVWAIRDGUZEFMXYTJQ
Part 2:  982
